In [ ]:
* For normalization, we shouldn't normalize the two alleles separately. 
    Although practically it won't really make a difference. 
    I would first find np.mean(list(data["x1"])+list(data["x2"]) 
                               and same for stddev and 
    Use those for ZNorm using same mean and stddev for each allele
* What is Locus_detail in nonlinearRegression?

* For the real thing, this should be an option added to the existing regression code. 
  e.g. you can add options 
        "--alleles" to specify that the input genotypes are in "allele1,allele2" format 
        and "--quadratic " to specify you also like to do the quadratic regression 
                    (which will output some extra columns)
                               
* You should also modify the existing linear regression function to handle the case when 
        the alleles are specified separately ("--alleles" above). 
        Then you can run both linear regression and nonlienar regression in same script.
                               
* You can have both those functions 
            return the AIC and we can use those to determine the best model 
            or shouldn't be that hard to also have an anova test for this.
                               
                               
                               
                               

In [ ]:

    
    # For each gene:
    # Pull out STRs within distance of gene ends
    # For each STR - gene pair, get expr, str for samples with data and do linreg
    PROGRESS("Expression annotation size %s "%str(expr_annot.shape))
    for i in range(expr_annot.shape[0]):
        gene = expr_annot.index.values[i]
        PROGRESS(" Getting data for %s"%gene)
        start = expr_annot["gene.start"].values[i]
        end = expr_annot["gene.stop"].values[i]
        cis_strs = strgt[(strgt["start"] >= (start-DISTFROMGENE)) & (strgt["start"] <= (end+DISTFROMGENE))]
        PROGRESS("%s STRs tested \n"%str(cis_strs.shape[0]))
       
        if PERMUTE_EXPR:
            expr[gene] = random.sample(list(expr[gene].values), expr.shape[0])
        y = pd.DataFrame({"expr":list(expr.loc[:, gene])})
        y.index = str_samples
#	print cis_strs
        for j in range(cis_strs.shape[0]):
            # Get STR data
            locus_str = cis_strs.iloc[[j],:][str_samples].transpose()
            locus_str.index = str_samples
            locus_str.columns = ["STR_%s"%(cis_strs["start"].values[j])]
            test_str=locus_str.columns[0]
            
            if QUAD:
                if ALLELE:
                    locus_str['x1'] = locus_str[test_str].apply(lambda x: x.split(',')[0] )
                    locus_str['x2'] = locus_str[test_str].apply(lambda x: x.split(',')[1] )
                    locus_str['locus'] = locus_str[test_str].apply(lambda x: int(x.split(',')[0]) + int(x.split(',')[1]) )
                    #NA|NA as genotype
                    samples_to_keep = [str_samples[k] for k in range(len(str_samples)) if (str(locus_str['x1'].values[k]) != "NA")and(str(locus_str['x2'].values[k]) != "NA")]
                    locus_str = locus_str.loc[samples_to_keep,:]
                    Locus_str = locus_str.join(locus_y)[['expr','x1','x2','locus']]
                    m = np.mean(list(locus_str["x1"].astype(int))+list(locus_str["x2"].astype(int))) 
                    sd= math.sqrt(np.var(locus_str["x1"].astype(int) + locus_str["x2"].astype(int) ))
                else:
                    sys.exit(1)      #This should not happen
            else:
            #None as genotype
                samples_to_keep = [str_samples[k] for k in range(len(str_samples)) if str(locus_str.iloc[:,0].values[k]) != "None" ]
                locus_str = locus_str.loc[samples_to_keep,:]
            # Filter            
            PROGRESS("Initial sample size:  %s"%str(len(locus_str.iloc[:,0])))
            locus_y = y.loc[samples_to_keep,:]
            #removing samples with outlier genotypes (> mean+/-5*sd)
            O1,O2 = GetOutlierLimit(map(float,locus_str.iloc[:,0].values))
            samples_to_keep1 = [samples_to_keep[k] for k in range(len(samples_to_keep)) if float(locus_str.iloc[:,0].values[k])<O1 and float(locus_str.iloc[:,0].values[k])>O2]
            locus_str = locus_str.loc[samples_to_keep1,:]
            locus_y = y.loc[samples_to_keep1,:]
            
            #ignore locus if number of #genotype <3
            if len(set(locus_str.iloc[:,0].values)) < MINGENOTYPE: 
                print locus_str.columns[0], ' skipped ................'
                continue
            PROGRESS('After clean up... %s'%str(locus_y.shape))
          
            # Run regression
            beta, beta_se, p = LinearRegression(map(float,locus_str.iloc[:,0].values), locus_y["expr"].values, norm=NORM, minsamples=MINSAMPLES)
            if QUAD:
                alpha, alpha_se, alpha_pval, beta, beta_se, beta_pval = nonlinearRegression(Locus_detail, norm=NORM, minsamples=MINSAMPLES)
                if alpha is not None and beta is not none:
                    f.write(' '.join([CHROM, gene, test_str, str(alpha), str(alpha_se), str(alpha_pval), str(beta), str(beta_se), str(beta_pval)])) 
             #define another file for quad regression output f1 , not f                  
#
            # Write output
            if beta is not None:
                str_start = cis_strs["start"].values[j]
                f.write("\t".join(map(str, [gene, CHROM, "STR_%s"%str_start, str_start, len(str_samples)-locus_str.shape[0], "A", "G", 0, beta, beta_se, -1, p]))+"\n")
#        break
    f.close()

